In [1]:
import seirmo

model = seirmo.SEIRModel()

## Visualise data

In [32]:
# Scale data in units of 100k
data = seirmo.DatasetLibrary().french_flu()
test_data = data.rename(columns={'time_index': 'Time', 'inc': 'Incidence Number'})
test_data['Incidence Number'] = test_data['Incidence Number'] / 1e6

fig = seirmo.plots.IncidenceNumberPlot()
fig.add_data(test_data.loc[0:30])
fig.show()

## Infer parameters

In [36]:
import numpy as np
import pints

# Create inverse problem
model.set_outputs(['Incidence'])
problem = pints.SingleOutputProblem(
    model=model, 
    times=test_data['Time'].to_numpy(),
    values=test_data['Incidence Number'].to_numpy())
log_likelihood = pints.GaussianLogLikelihood(problem)
log_prior = pints.ComposedLogPrior(
    pints.UniformLogPrior(0, 1),
    pints.UniformLogPrior(0, 1),
    pints.UniformLogPrior(0, 1),
    pints.UniformLogPrior(0, 1),
    pints.UniformLogPrior(0, 1),
    pints.UniformLogPrior(0, 1),
    pints.UniformLogPrior(0, 1),
    pints.UniformLogPrior(0, 1))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Run inference
n_runs = 1
transformations = pints.LogTransformation(log_posterior.n_parameters())

estimates = np.empty(shape=(n_runs, log_posterior.n_parameters()))
for run_id in range(n_runs):
    initial_parameters = log_prior.sample()
    opt = pints.OptimisationController(
        function=log_posterior,
        x0=initial_parameters,
        method=pints.CMAES,
        transform=transformations)

    estimates[run_id] = opt.run()

Maximising LogPDF
Using Covariance Matrix Adaptation Evolution Strategy (CMA-ES)
Running in sequential mode.
Population size: 10
Iter. Eval. Best      Time m:s
0     10    -1295.991   0:01.0
1     20    -1233.902   0:02.0
2     30    -1148.346   0:03.0
3     40    -1076.335   0:04.0
20    210    1148.955   0:16.0
40    410    1152.511   0:28.7
60    610    1155.307   0:42.1
80    810    1155.816   0:55.7
100   1010   1156.009   1:13.1
120   1210   1156.142   1:26.3
140   1410   1156.425   1:39.9
160   1610   1157.825   1:52.8
180   1810   1164.129   2:02.7
200   2010   1166.806   2:14.3
220   2210   1167.76    2:24.6
240   2410   1167.951   2:33.6
260   2610   1168.001   2:45.4
280   2810   1168.07    2:56.3
300   3010   1168.228   3:06.4
320   3210   1168.295   3:16.8
340   3410   1168.355   3:26.0
360   3610   1168.366   3:35.1
380   3810   1168.371   3:44.1
400   4010   1168.382   3:54.1
420   4210   1168.39    4:03.4
440   4410   1168.403   4:14.0
460   4610   1168.423   4:22.5
480

KeyboardInterrupt: 

In [35]:
%%timeit
model.simulate(np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]), test_data['Time'].to_numpy())

14.9 ms ± 754 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
initial_parameters[0]

array([1.26828906, 2.1567411 , 3.94264152, 2.14637825, 8.87084302,
       3.93934852, 9.05002   , 0.7038902 ])

In [27]:
import myokit

# Create model
myokit_model = myokit.Model()

# Create states and parameters
comp = myokit_model.add_component('myokit')
s = comp.add_variable('susceptible')
e = comp.add_variable('exposed')
i = comp.add_variable('infected')
r = comp.add_variable('recovered')
incubation_rate = comp.add_variable('incubation_rate')
infection_rate = comp.add_variable('infection_rate')
recovery_rate = comp.add_variable('recovery_rate')
time = comp.add_variable('time')

# Bind time variable
time.set_binding('time')

# Set parameter values
infection_rate.set_rhs(0.1)
incubation_rate.set_rhs(0.1)
recovery_rate.set_rhs(0.1)

# Set RHS of states
s.promote(1.26828906)
s.set_rhs(
    myokit.Multiply(
        myokit.Multiply(
            myokit.PrefixMinus(myokit.Name(infection_rate)),
            myokit.Name(s)),
        myokit.Name(i)))
e.promote(2.1567411)
e.set_rhs(
    myokit.Minus(
        myokit.Multiply(
            myokit.Multiply(
                myokit.Name(infection_rate),
                myokit.Name(s)),
            myokit.Name(i)),
        myokit.Multiply(
            myokit.Name(incubation_rate),
            myokit.Name(e))))
i.promote(3.94264152)
i.set_rhs(
    myokit.Minus(
        myokit.Multiply(
            myokit.Name(incubation_rate),
            myokit.Name(e)),
        myokit.Multiply(
            myokit.Name(recovery_rate),
            myokit.Name(i))))
r.promote(2.14637825)
r.set_rhs(
    myokit.Multiply(
        myokit.Name(recovery_rate),
        myokit.Name(i)))
time.set_rhs(0)


print(myokit_model.code())

[[model]]
# Initial values
myokit.susceptible =  1.26828906000000008
myokit.exposed     = 2.1567411
myokit.infected    =  3.94264152000000001
myokit.recovered   =  2.14637825000000015

[myokit]
dot(exposed) = infection_rate * susceptible * infected - incubation_rate * exposed
incubation_rate =  3.93934851999999980
dot(infected) = incubation_rate * exposed - recovery_rate * infected
infection_rate =  8.87084302000000058
dot(recovered) = recovery_rate * infected
recovery_rate = 9.05002
dot(susceptible) = -infection_rate * susceptible * infected
time = 0 bind time




In [29]:
sim = myokit.Simulation(myokit_model)

In [30]:
output = sim.run(test_data['Time'].max()+1)

# Get e.g. susceptibles
s = np.array(outputs['myokit.susceptible'])

-11, 4.84087089284728e-11, -1.8378407985332386e-12, -3.1905158948323907e-12, -2.1190675126354322e-13, -1.8286704607634126e-13])),
         ('myokit.recovered',
          array('d', [2.14637825, 2.154960305910165, 2.1635281789472733, 2.181450517015848, 2.19931795651155, 2.217126264411587, 2.2499475069706008, 2.345720237403279, 2.392522733346934, 2.4389055421394583, 2.48487611892039, 2.5540544207930114, 2.622315968552433, 2.689685353203739, 2.8021871219219228, 2.9122726761516033, 3.020032826012917, 3.1255510239438, 3.2289048204246735, 3.330166663765295, 3.429404420607467, 3.5266818033898124, 3.622058759496006, 3.7155918266296597, 3.80733445075852, 3.8973372666493162, 3.9856483444693067, 4.0723134070401885, 4.157376022027728, 4.283136987334921, 4.405459889984354, 4.524472768713757, 4.6402954302362645, 4.753040638521883, 4.8628150857132, 4.9697199404321335, 5.073851209807061, 5.175300099948027, 5.333835160636324, 5.485931505030654, 5.631908881553498, 5.772065289320053, 5.906679372042032, 6

In [31]:
%%timeit

sim.run(test_data['Time'].max()+1)

1.33 ms ± 54.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
class ForwardModel(pints.ForwardModel):
    def __init__(self):
        super(ForwardModel, self).__init__()

        # Create model
        model = myokit.Model()

        # Create states and parameters
        comp = myokit_model.add_component('myokit')
        s = comp.add_variable('susceptible')
        e = comp.add_variable('exposed')
        i = comp.add_variable('infected')
        r = comp.add_variable('recovered')
        incubation_rate = comp.add_variable('incubation_rate')
        infection_rate = comp.add_variable('infection_rate')
        recovery_rate = comp.add_variable('recovery_rate')
        time = comp.add_variable('time')

        # Bind time variable
        time.set_binding('time')

        # Set parameter values
        infection_rate.set_rhs(8.87084302)
        incubation_rate.set_rhs(3.93934852)
        recovery_rate.set_rhs(9.05002)

        # Set RHS of states
        s.promote(1.26828906)
        s.set_rhs(
            myokit.Multiply(
                myokit.Multiply(
                    myokit.PrefixMinus(myokit.Name(infection_rate)),
                    myokit.Name(s)),
                myokit.Name(i)))
        e.promote(2.1567411)
        e.set_rhs(
            myokit.Minus(
                myokit.Multiply(
                    myokit.Multiply(
                        myokit.Name(infection_rate),
                        myokit.Name(s)),
                    myokit.Name(i)),
                myokit.Multiply(
                    myokit.Name(incubation_rate),
                    myokit.Name(e))))
        i.promote(3.94264152)
        i.set_rhs(
            myokit.Minus(
                myokit.Multiply(
                    myokit.Name(incubation_rate),
                    myokit.Name(e)),
                myokit.Multiply(
                    myokit.Name(recovery_rate),
                    myokit.Name(i))))
        r.promote(2.14637825)
        r.set_rhs(
            myokit.Multiply(
                myokit.Name(recovery_rate),
                myokit.Name(i)))
        time.set_rhs(0)

        # Instantiate simulator
        self._sim = myokit.Simulation(model)

    def n_parameters(self):
        return 7

    def simulate(self, parameters, times):
        # Reset
        self._sim.reset()

        # Unpack parameters

        # Set initial states

        # Set parameters

